In [1]:
from imutils.video import VideoStream
from imutils.video import FPS
import argparse
import imutils
import time
import cv2
import pandas as pd

In [2]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-v", "--video", type=str,
	help="path to input video file")
ap.add_argument("-t", "--tracker", type=str, default="csrt",
	help="OpenCV object tracker type")
args = vars(ap.parse_known_args()[0])

In [3]:
(major, minor) = cv2.__version__.split(".")[:2]
if int(major) == 3 and int(minor) < 3:
	tracker = cv2.Tracker_create(args["tracker"].upper())
else:
	OPENCV_OBJECT_TRACKERS = {
		"csrt": cv2.TrackerCSRT_create,
		"kcf": cv2.TrackerKCF_create,
		# "boosting": cv2.TrackerBoosting_create,
		"mil": cv2.TrackerMIL_create,
		# "tld": cv2.TrackerTLD_create,
		# "medianflow": cv2.TrackerMedianFlow_create,
		# "mosse": cv2.TrackerMOSSE_create
	}
	tracker = OPENCV_OBJECT_TRACKERS[args["tracker"]]()

mil
tld

In [15]:
# loop over frames from the video stream
df = pd.DataFrame(columns = ['label', 'time', 'x', 'y', 'rad_pos'])
initBB = None
vs = cv2.VideoCapture("../test_videos/test_4.mp4")
fps = vs.get(5)
circle_coords = []
while True:
	# grab the current frame, then handle if we are using a
	# VideoStream or VideoCapture object
	frame = vs.read()[1]
	if frame is None:
		break
	# resize the frame (so we can process it faster) and grab the
	# frame dimensions
	(H, W) = frame.shape[:2]
		# check to see if we are currently tracking an object
	if initBB is not None:
		# grab the new bounding box coordinates of the object
		(success, box) = tracker.update(frame)
		# check to see if the tracking was a success
		if success:
			(x, y, w, h) = [int(v) for v in box]
			# cv2.rectangle(frame, (x, y), (x + w, y + h),(0, 255, 0), 2)
			curr_time = vs.get(cv2.CAP_PROP_POS_MSEC)
			df.loc[df.shape[0]] = ['Center', curr_time/1000, x+w/2, y+h/2, (x+w/2)**2 + (y+h/2)**2]
			circle_coords.append((int(x+w/2),int(y+h/2)))
			for xy in circle_coords:
				cv2.circle(frame, xy, 1, (0, 255, 0), 0.5)
		# update the FPS counter
		fps.update()
		fps.stop()
		# initialize the set of information we'll be displaying on
		# the frame
		info = [
			("Tracker", args["tracker"]),
			("Success", "Yes" if success else "No"),
			("FPS", "{:.2f}".format(fps.fps())),
		]
		# loop over the info tuples and draw them on our frame
		for (i, (k, v)) in enumerate(info):
			text = "{}: {}".format(k, v)
			cv2.putText(frame, text, (10, H - ((i * 20) + 20)),
				cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
		# show the output frame
	cv2.imshow("Frame", frame)
	
	key = cv2.waitKey(1) & 0xFF
	# if the 's' key is selected, we are going to "select" a bounding
	# box to track
	if key == ord("s"):
		# select the bounding box of the object we want to track (make
		# sure you press ENTER or SPACE after selecting the ROI)
		initBB = cv2.selectROI("Frame", frame, fromCenter=False,
			showCrosshair=True)
		# start OpenCV object tracker using the supplied bounding box
		# coordinates, then start the FPS throughput estimator as well
		tracker.init(frame, initBB)
		fps = FPS().start()
			# if the `q` key was pressed, break from the loop
	elif key == ord("q"):
		break
vs.release()
# close all windows
cv2.destroyAllWindows()

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Argument 'thickness' is required to be an integer
>  - Argument 'thickness' is required to be an integer


: 

In [13]:
df

,label,time,x,y,rad_pos
0,Center,2.702700,347.5,624.5,510756.50
1,Center,2.736066,348.0,625.0,511729.00
2,Center,2.769433,349.0,625.0,512426.00
3,Center,2.802800,349.0,626.0,513677.00
4,Center,2.836167,349.0,626.5,514303.25
...,...,...,...,...,...
705,Center,26.226200,350.0,636.5,527632.25
706,Center,26.259567,350.0,636.5,527632.25
707,Center,26.292933,350.5,637.0,528619.25
708,Center,26.326300,350.0,636.5,527632.25
